In [2]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import urllib.request as urlr
import cartopy.io.shapereader as shpreader
import geopandas as gpd
import glob
from scipy import spatial
from scipy.interpolate import RegularGridInterpolator

from plasticparcels.utils import distance, get_coords_from_polygon

In [ ]:
# Function definitions
def create_coastal_mpw_jambeck_release_map(mask_coast_filepath, coords_filepath, gpw_filepath,
                                           distance_threshhold=50., grid_range=0.083,
                                           gpw_column_name='Population Density, v4.11 (2000, 2005, 2010, 2015, 2020): 2.5 arc-minutes',
                                           gpw_raster_number=4,
                                           jambeck_filepath=None,
                                           jambeck_url='https://www.science.org/doi/suppl/10.1126/science.1260352/suppl_file/1260352_supportingfile_suppl._excel_seq1_v1.xlsx',
                                           jambeck_url_backup='http://jambeck.engr.uga.edu/wp-content/uploads/2015/01/JambeckSData.xlsx'
                                           ):
    """
    Description
    ----------

    A function to create a particle release map based on the Mismanaged Plastice Waste data from Jambeck et al. (2015).
    We first load the Natural Earth coastline vector dataset. For each vertex in the vector dataset, we identify any
    associated coastal cell within distance_threshhold km. We then assign a population density from the GPW dataset,
    computed as the maximum population density within grid_range degrees. We then left-join the Jambeck mismanaged
    plastic waste data to compute the total mismanaged plastic waste in kg/day per coastal grid cell.


    Parameters
    ----------

    mask_coast_filepath : str
        File path to the coastal mask generated in ...py TODO
    coords_filepath : str
        File path to the model grid coordinates.
    gpw_filepath : str
        File path to the 'Gridded Population of the World (GPW)' dataset.
    distance_threshhold : float, optional
        The threshhold distance used to find coastal grid cells associated to Natural Earth vector vertices, by default 50
    grid_range : float, optional
        The approximate grid width to search for the maximum population density surrounding a coastal grid cell, by default 0.083
    gpw_column_name : str, optional
        The column name of the GPW dataset that corresponds to the population density, by default 'Population Density, v4.11 (2000, 2005, 2010, 2015, 2020): 2.5 arc-minutes'
    gpw_raster_number : int, optional
        The raster number of the dataset to use. When using the GPW v4.11 data, raster = 4 provides the 2020 population density.
    jambeck_filepath : str, optional
        The filepath to the Jambeck dataset if downloaded manually.
    jambeck_url : str, optional
        The URL of the Jambeck dataset from the Science article.
    jambeck_url_backup : str, optional
        The URL of the Jambeck dataset from Jenna Jambeck's website.

    Returns
    -------
    coastal_density_mpw_df
        A pandas dataframe containing the coastal grid cells, associated countries, and associated mismanaged plastic waste in kg/day.
    """

    # Open the Natural Earth coastline vector dataset
    shpfilename = shpreader.natural_earth(resolution='50m',
                                          category='cultural',
                                          name='admin_0_countries')

    reader = shpreader.Reader(shpfilename)
    countries = reader.records()

    # Open the GPW dataset, and set NaNs to zeros (i.e. zero population density in the ocean)
    gpw = xr.open_dataset(gpw_filepath)
    gpw = gpw.fillna(0.)

    # Load in coast mask and model coordinates
    data_mask_coast = xr.open_dataset(mask_coast_filepath)
    coords = xr.open_dataset(coords_filepath, decode_cf=False)

    lats_coast = data_mask_coast['lat'].data[np.where(data_mask_coast['mask_coast'])]
    lons_coast = data_mask_coast['lon'].data[np.where(data_mask_coast['mask_coast'])]

    # Compute the area of each grid cell
    cell_areas = coords['e1t'][0] * coords['e2t'][0]/10e6  # in km**2
    coastal_cell_areas = cell_areas.data[np.where(data_mask_coast['mask_coast'])]

    # Loop through all countries from Natural Earth dataset
    coastal_density_list = []
    for country in countries:
        # Country information
        continent = country.attributes['CONTINENT']
        region_un = country.attributes['REGION_UN']
        subregion = country.attributes['SUBREGION']
        country_name = country.attributes['NAME_LONG']

        # Extract longitude and latitude of the coastal point
        country_coords = get_coords_from_polygon(country.geometry)
        country_lons, country_lats = country_coords[:, 0], country_coords[:, 1]

        # Loop through country points to find coastal points within distance_threshhold km
        all_coastal_indices = []
        for i in range(len(country_lons)):
            distances = distance(np.repeat(country_lons[i], len(lons_coast)), np.repeat(country_lats[i], len(lats_coast)), lons_coast, lats_coast)
            coastal_indices = np.where(distances <= distance_threshhold)[0]  # Coastal indices are those that are within the thresshold distance
            all_coastal_indices.append(coastal_indices)

        # Concatenate into one list and identify the unique coastal cells
        all_coastal_indices = np.unique(np.hstack(all_coastal_indices))

        # For all coastal points assigned to the country, find the maximum population density around that point
        country_coastal_density_list = []
        for i in range(len(all_coastal_indices)):
            coastal_id = all_coastal_indices[i]
            coastal_point = [lons_coast[coastal_id], lats_coast[coastal_id]]
            coastal_cell_area = coastal_cell_areas[coastal_id]

            # Compute the population density as the maximum population density around the coastal cell
            # Because the grid is ordered longitude ascending, latitude descending, the slice ordering is swapped for lat
            population_density = gpw[gpw_column_name].sel(longitude=slice(coastal_point[0] - grid_range, coastal_point[0] + grid_range),
                                                          latitude=slice(coastal_point[1] + grid_range, coastal_point[1] - grid_range),
                                                          raster=gpw_raster_number).max()

            population_density = np.float32(population_density)

            country_coastal_density_list.append({'Continent': continent,
                                                 'Region': region_un,
                                                 'Subregion': subregion,
                                                 'Country': country_name,
                                                 'Longitude': coastal_point[0],
                                                 'Latitude': coastal_point[1],
                                                 'Area[km2]': coastal_cell_area,
                                                 'PopulationDensity': population_density})

        country_coastal_density_df = pd.DataFrame.from_records(country_coastal_density_list)
        coastal_density_list.append(country_coastal_density_df)

    # Concatenate all the information into one dataframe
    coastal_density_df = pd.concat(coastal_density_list)

    # Open the Jambeck dataset
    if jambeck_filepath is not None:
        MPW = pd.read_excel(jambeck_filepath, 'Jambeck et al. (2014)')
    else:
        try:
            socket = urlr.urlopen(jambeck_url)
        except:
            socket = urlr.urlopen(jambeck_url_backup)
        spreadsheet_from_url = pd.ExcelFile(socket.read())
        MPW = pd.read_excel(spreadsheet_from_url, 'Jambeck et al. (2014)')

    # Perform some data cleaning steps
    # 1. Rename the columns to make it easier to work with
    MPW = MPW.rename(columns={'Economic status1': 'Economic status',
                              'Mismanaged plastic waste [kg/person/day]7': 'Mismanaged plastic waste [kg/person/day]'})

    # 2. Set the bottom rows with black info to blank strings and only keep data with valid country names
    MPW = MPW.fillna('')
    MPW = MPW[MPW['Country'] != '']

    # 3. Remove all sub/superscripts and standardise the 'and' and ampersands.
    MPW['Country'] = MPW['Country'].replace(regex='[0-9]', value='').str.replace('&', 'and')

    # 4. Manually rename some countries to match the Natural Earth Dataset
    MPW['Country'] = MPW['Country'].replace('Brunei', 'Brunei Darussalam')
    MPW['Country'] = MPW['Country'].replace('Curacao', 'Curaçao')
    MPW['Country'] = MPW['Country'].replace('Congo, Dem rep. of', 'Democratic Republic of the Congo')
    MPW['Country'] = MPW['Country'].replace('Korea, North', 'Dem. Rep. Korea')
    MPW['Country'] = MPW['Country'].replace('Korea, South (Republic of Korea)', 'Republic of Korea')
    MPW['Country'] = MPW['Country'].replace('Burma/Myanmar', 'Myanmar')
    MPW['Country'] = MPW['Country'].replace('Micronesia', 'Federated States of Micronesia')
    MPW['Country'] = MPW['Country'].replace('Faroe Islands', 'Faeroe Islands')
    MPW['Country'] = MPW['Country'].replace('Falkland Islands', 'Falkland Islands / Malvinas')
    MPW['Country'] = MPW['Country'].replace('Cote d\'Ivoire', 'Côte d\'Ivoire')
    MPW['Country'] = MPW['Country'].replace('East Timor', 'Timor-Leste')
    MPW['Country'] = MPW['Country'].replace('Russia', 'Russian Federation')
    MPW['Country'] = MPW['Country'].replace('Saint Pierre', 'Saint Pierre and Miquelon')
    MPW['Country'] = MPW['Country'].replace('Congo Rep of', 'Republic of the Congo')
    MPW['Country'] = MPW['Country'].replace('Palestine (Gaza Strip is only part on the coast)', 'Palestine')
    MPW['Country'] = MPW['Country'].replace('Saint Maarten, DWI', 'Sint Maarten')
    MPW['Country'] = MPW['Country'].replace('USVI', 'United States Virgin Islands')
    MPW['Country'] = MPW['Country'].replace('Sao Tome and Principe', 'São Tomé and Principe')

    # TODO: Sort out these countries...
    # Below commented out because it obviously doubles up rows when left-joining.
    # MPW['Country'] = MPW['Country'].replace('Northern Cyprus', 'Cyprus') # Combines the two sets
    # MPW['Country'] = MPW['Country'].replace('Gibraltar', 'United Kingdom') # Add to UK
    # MPW['Country'] = MPW['Country'].replace('French Guiana', 'France')
    # MPW['Country'] = MPW['Country'].replace('Guadeloupe', 'France')
    # MPW['Country'] = MPW['Country'].replace('Martinique', 'France')
    # MPW['Country'] = MPW['Country'].replace('Christmas Island', 'Australia')
    # MPW['Country'] = MPW['Country'].replace('Reunion', 'France')
    # MPW['Country'] = MPW['Country'].replace('Netherlands Antilles', 'Netherlands')

    # Combine the coastal cell density data with the mismanaged plastic waste data, performing a left-join on the country column
    coastal_density_mpw_df = pd.merge(coastal_density_df, MPW[['Country', 'Economic status', 'Mismanaged plastic waste [kg/person/day]']], on="Country", how="left")

    # Compute the mismanaged plastic waste in units of kg/day
    coastal_density_mpw_df['MPW_Cell'] = coastal_density_mpw_df['Area[km2]']*coastal_density_mpw_df['PopulationDensity']*coastal_density_mpw_df['Mismanaged plastic waste [kg/person/day]']

    return coastal_density_mpw_df

In [ ]:
mask_land_filepath = '/Users/denes001/Research/Projects/PlasticParcels/PlasticParcels/data/output_data/masks/mask_land_NEMO0083.nc'
mask_coast_filepath = '/Users/denes001/Research/Projects/PlasticParcels/PlasticParcels/data/output_data/masks/mask_coast_NEMO0083.nc'
coords_filepath = '/Users/denes001/Research/Projects/PlasticParcels/PlasticParcels/data/input_data/MOi/domain_ORCA0083-N006/coordinates.nc'

# Create coastal release data
gpw_filepath = '/Users/denes001/Research/Projects/PlasticParcels/PlasticParcels/data/release/GPWv4/gpw-v4-population-density-rev11_totpop_2pt5_min_nc/gpw_v4_population_density_rev11_2pt5_min.nc'